In [ ]:
gpu='cuda:0'
get_prompt_structure = lambda:'an award winning photograph of a %s'
get_prompt_structure = lambda:'an award winning photograph of a %s ' + rp.random_choice('in times square', 'in the deep jungle')
# get_prompt_structure = lambda:'3d pixar style render animation of a %s'

In [ ]:
from rp import *
base_notebook_path = "/workspace/Diffusion-Illusions/hidden_characters_for_colab__advanced_miku_BASE.ipynb"
base_notebook_code = extract_code_from_ipynb(base_notebook_path)
seed_nummy=rp.random_int(0,1000000)

In [ ]:
HEADLESS=True
for trial_num in range(10000):
    trial_num+=0 #So we can preview one of the algos faster
    # import random
    # random.seed(trial_num) #For picking the prompts
    
    BIG=True
    BIG=False
    
    USE_SDXL=True
    # USE_SDXL=False
    
    
    SAVE_TITLE='auto_test_1'
    
    #FOR SDS
    model_name="CompVis/stable-diffusion-v1-4"
    model_name="runwayml/stable-diffusion-v1-5"
    
    SEED=seed_nummy
    
    AUTO_WEIGHT=True
    
    #################################################################
    
    # prompt_a = 'a photo of the deep jungle'
    # prompt_b = 'a photo of the deep jungle'
    # prompt_c = 'a photo of the deep jungle'
    # prompt_d = 'a photo of the deep jungle'
    # prompt_z = 'the word "DANGER!"'


    method_names = [
        'scoredistill_sd15',
        'dreamtarget_noautoweight_sdxl',
        'dreamtarget_autoweight_sd15',
        'dreamtarget_autoweight_sdxl',
    ]
    method_name = method_names[trial_num%len(method_names)]

    if not trial_num%len(method_names):
        #Switch the prompts once in a while
        import random
        random.seed(rp.gtoc())
        prompt_structure=get_prompt_structure()
        possible_subjects = 'bird, cat, cow, dog, horse, sheep, aeroplane, bicycle, boat, bus, car, motorbike, train, bottle, chair, dining table, potted plant, sofa, tv monitor'.split(', ')
        subjects = rp.random_batch(possible_subjects,5)
        prompts_abcdz =  [prompt_structure%subject for subject in subjects]
        prompt_a, prompt_b, prompt_c, prompt_d, prompt_z = prompts_abcdz
        
        SAVE_TITLE='_'.join(subjects)
    
    import icecream
    icecream.ic(prompt_a,prompt_b,prompt_c,prompt_d,prompt_z)
    
    
    negative_prompt = 'blurry blurry blurry blurry blurry blurry grayscale, blurry'
    
    #################################################################
    
    SMOOTH_PROB=0
    MSE_COEF = 1
    
    
    ZI_OVERRIDE=None
    # ZI_OVERRIDE=rp.load_image("https://m.media-amazon.com/images/I/51nHtpT5hEL._AC_UF894,1000_QL80_.jpg",use_cache=True) ; WEIGHTS_LIST=[1,1,1,1,.5]
    # ZI_OVERRIDE=rp.inverted_image(rp.load_image("https://i.imgur.com/BlnAQxm.png",use_cache=True)) ; WEIGHTS_LIST=[1,1,1,1,.5] #rig poem
    # ZI_OVERRIDE=rp.load_image("https://i.imgur.com/F006VHD.png",use_cache=True) ; #WEIGHTS_LIST=[1,1,1,1,.5]
    # ZI_OVERRIDE=rp.inverted_image(rp.load_image("https://m.media-amazon.com/images/I/612+XnLD1RL._AC_UF894,1000_QL80_.jpg",use_cache=True)) ; WEIGHTS_LIST=[1,1,1,1,1/2] #Pentagram
    # ZI_OVERRIDE=rp.load_image("https://cdn2.vectorstock.com/i/1000x1000/97/86/colorful-rainbow-swirl-vector-1549786.jpg",use_cache=True) ; WEIGHTS_LIST=[1,1,1,1,1/2] #Rainbow Swirl
    # ZI_OVERRIDE=rp.load_image("https://upload.wikimedia.org/wikipedia/en/e/ed/Nyan_cat_250px_frame.PNG",use_cache=True) ; WEIGHTS_LIST=[1,1,1.5,2,2] #Nyan Cat
    # ZI_OVERRIDE=rp.load_image("https://i.imgur.com/IrkQ4f3.jpeg",use_cache=True) ; WEIGHTS_LIST=[1,1,1,1,.5]
    
    START_HERE=hash(object())

    #So it doesn't crash - it needs these defined. they'll be replaced from sss.
    GUIDANCE=20
    WEIGHTS_LIST=[1,1,1,1,1]
    STEPS=3000
    MSSSIM_COEF=0 #SPEEDUP
    NUM_IM2IM_STEPS=20 #SPEEDUP

    

    
    SAVE_DIR='cheapo_formal_loggers/'+method_name
    
    if method_name=='dreamtarget_autoweight_sdxl' or method_name=='dreamtarget_noautoweight_sdxl':
        print(method_name);rp.sleep(1)
        USE_SDXL=True
        SDS_ITERS=1
        AUTO_WEIGHT=method_name=='dreamtarget_autoweight_sdxl'
        WEIGHTS_LIST=[1,1,1,1,1] if method_name=='dreamtarget_autoweight_sdxl' else [1,1,1,1,2]
        sss=[
            START_HERE, #A bookmark
            dict(
                GUIDANCE=20,
                STEPS=3000,
                MSSSIM_COEF=0, #SPEEDUP
                NUM_IM2IM_STEPS=40, #SPEEDUP
            ),
            *[1,.9,.9,.8,.8],
            dict(GUIDANCE=30,steps=2000),
            dict(MSSSIM_COEF=.2),
            *[.7,.5,.7],
            dict(STEPS=3000),
            *[.5]*1,
        ]
        
        
    if method_name=='dreamtarget_autoweight_sd15':
        print(method_name);rp.sleep(1)
        SDS_ITERS=1
        USE_SDXL=False
        AUTO_WEIGHT=True
        sss=[
            START_HERE, #A bookmark
            dict(
                GUIDANCE=15,
                # WEIGHTS_LIST=[1,1,1,1,1] if ZI_OVERRIDE is None else [1,1,1,1,1],
                WEIGHTS_LIST=[1,1,1,1,1],
                STEPS=3000,
                MSSSIM_COEF=0, #SPEEDUP
                NUM_IM2IM_STEPS=300, #SPEEDUP
            ),
            *[1,.9,.9,.8,.8],#.5,.3],#,1,.7,.5,.3],
            dict(GUIDANCE=20),
            *[.7,.5]*1,
            dict(MSSSIM_COEF=.2),
            *[.7,.5]*1,
            # *[.5]*2,
            dict(STEPS=5000),
            *[.5]*1,
        ]



    
    if method_name=='scoredistill_sd15':
        print(method_name);rp.sleep(1)
        SDS_ITERS=4000
        AUTO_WEIGHT=False
        sss=[
            START_HERE, #A bookmark
            dict(
                GUIDANCE=20,
                WEIGHTS_LIST=[1,1,1,1,2],
                STEPS=3000,
                MSSSIM_COEF=0, #SPEEDUP
                NUM_IM2IM_STEPS=20, #SPEEDUP
            ),
            
            dict(GUIDANCE=30,steps=2000),
            dict(MSSSIM_COEF=.2),
            dict(STEPS=2500),
            .7,
            dict(STEPS=4000),
            .5,
        ]
        USE_SDXL=True



    
    exec(base_notebook_code)